In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [4]:
ind =dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes',]]
dep=dataset[['classification_yes']]

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(ind, dep, test_size=0.30, random_state=0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test= sc.transform(x_test)

In [7]:
from sklearn.svm import SVC

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','poly','rbf','sigmoid'],
              'gamma': ['scale', 'auto'],
               'C':[10,100,1000,2000,3000]}

grid = GridSearchCV (SVC(probability=True), param_grid, refit=True, verbose=3, cv=5,  n_jobs=-1, scoring='f1_weighted')

grid.fit(x_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Gowri\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [9]:
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test,grid_predictions) 



In [10]:
from sklearn.metrics import classification_report
clf_report= classification_report(y_test,grid_predictions) 


In [11]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test,grid_predictions, average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}: 0.9834018801410106


In [12]:
print("The confusion matrix:\n",cm)

The confusion matrix:
 [[45  0]
 [ 2 73]]


In [13]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        45
           1       1.00      0.97      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.99      0.98       120
weighted avg       0.98      0.98      0.98       120



In [14]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9997037037037036

In [15]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006004,0.004902,0.006265,0.003114,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.982221,0.982221,0.982221,0.982051,0.981894,0.982122,0.000132,25
1,0.006722,0.002688,0.009360,0.004180,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",1.000000,1.000000,0.964286,1.000000,0.981894,0.989236,0.014311,13
2,0.002003,0.004005,0.009964,0.004071,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.982221,1.000000,0.982051,1.000000,1.000000,0.992854,0.008752,3
3,0.004491,0.005500,0.007063,0.005798,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.982221,1.000000,0.982221,1.000000,1.000000,0.992888,0.008710,1
4,0.006999,0.004798,0.003314,0.003715,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.982221,0.982221,0.982221,0.982051,0.981894,0.982122,0.000132,25
5,0.010092,0.000072,0.002624,0.003755,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",1.000000,1.000000,0.964286,1.000000,0.981894,0.989236,0.014311,13
6,0.004705,0.005171,0.004393,0.005393,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.982221,1.000000,0.982051,1.000000,1.000000,0.992854,0.008752,3
7,0.008244,0.004125,0.002772,0.004640,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.982221,1.000000,0.982221,1.000000,1.000000,0.992888,0.008710,1
8,0.002001,0.004001,0.008899,0.003556,100,scale,linear,"{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}",0.982221,0.982221,0.982221,0.982051,0.981894,0.982122,0.000132,25
9,0.008705,0.004095,0.002192,0.004081,100,scale,poly,"{'C': 100, 'gamma': 'scale', 'kernel': 'poly'}",0.964572,1.000000,0.982051,1.000000,0.981894,0.985703,0.013290,17
